In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
sys.path.append("../../")

from src.models.sgpr.training import model_training_gpr
from src.models.sgpr.save_load_model import save_model_gpr, load_model_gpr
from src.models.sgpr.predicting import model_test_predictions_gpr
from src.misc import create_folder
# from seed_file import seed

seed = 2023
tuning_seed = 2023
data_label = "mimic"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_data_folder = join(fp_project_folder, "../", "data")
fp_processed_folder = join(fp_project_folder, "processed_data", data_label)
fp_output_data_folder = join(fp_data_folder, data_label)
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")

# Seed filepaths
fp_cur_tune_folder = join(fp_tuning, str(tuning_seed))
create_folder(fp_cur_tune_folder)
fp_cur_model_folder = join(fp_models, str(seed))
create_folder(fp_cur_model_folder)
fp_cur_predictions_folder = join(fp_predictions, str(seed))
create_folder(fp_cur_predictions_folder)

# Load Data

In [ ]:
split_dict = joblib.load(join(fp_processed_folder, "mimic_split_dict.joblib"))
predictors = ['ABPdias (mmHg)_mean Min0', 'RESP (bpm)_mean Min0', 'ABPmean (mmHg)_mean Min0', 'ABPsys (mmHg)_mean Min0', 'SpO2 (%)_mean Min0', 'HR (bpm)_mean Min0', 'ABPdias (mmHg)_std Min0', 'RESP (bpm)_std Min0', 'ABPmean (mmHg)_std Min0', 'ABPsys (mmHg)_std Min0', 'SpO2 (%)_std Min0', 'HR (bpm)_std Min0', 'ABPdias (mmHg)_mean Min1', 'RESP (bpm)_mean Min1', 'ABPmean (mmHg)_mean Min1', 'ABPsys (mmHg)_mean Min1', 'SpO2 (%)_mean Min1', 'HR (bpm)_mean Min1', 'ABPdias (mmHg)_std Min1', 'RESP (bpm)_std Min1', 'ABPmean (mmHg)_std Min1', 'ABPsys (mmHg)_std Min1', 'SpO2 (%)_std Min1', 'HR (bpm)_std Min1', 'ABPdias (mmHg)_mean Min2', 'RESP (bpm)_mean Min2', 'ABPmean (mmHg)_mean Min2', 'ABPsys (mmHg)_mean Min2', 'SpO2 (%)_mean Min2', 'HR (bpm)_mean Min2', 'ABPdias (mmHg)_std Min2', 'RESP (bpm)_std Min2', 'ABPmean (mmHg)_std Min2', 'ABPsys (mmHg)_std Min2', 'SpO2 (%)_std Min2', 'HR (bpm)_std Min2', 'ABPdias (mmHg)_mean Min3', 'RESP (bpm)_mean Min3', 'ABPmean (mmHg)_mean Min3', 'ABPsys (mmHg)_mean Min3', 'SpO2 (%)_mean Min3', 'HR (bpm)_mean Min3', 'ABPdias (mmHg)_std Min3', 'RESP (bpm)_std Min3', 'ABPmean (mmHg)_std Min3', 'ABPsys (mmHg)_std Min3', 'SpO2 (%)_std Min3', 'HR (bpm)_std Min3', 'ABPdias (mmHg)_mean Min4', 'RESP (bpm)_mean Min4', 'ABPmean (mmHg)_mean Min4', 'ABPsys (mmHg)_mean Min4', 'SpO2 (%)_mean Min4', 'HR (bpm)_mean Min4', 'ABPdias (mmHg)_std Min4', 'RESP (bpm)_std Min4', 'ABPmean (mmHg)_std Min4', 'ABPsys (mmHg)_std Min4', 'SpO2 (%)_std Min4', 'HR (bpm)_std Min4']

# Train SGPR

In [ ]:
for time_label, time_info_dict in tqdm(split_dict.items()):
    gpr = model_training_gpr(
        predictors=predictors, pred_cols=time_info_dict["outputs"], 
        train_df=time_info_dict["train_df"], valid_df = time_info_dict["valid_df"], 
        seed=seed, prop_inducing=0.001
    ) 
    save_model_gpr(
        model=gpr, name=f"gpr_{time_label}", 
        fp_checkpoints=fp_cur_model_folder, predictors=predictors)

# Prediction

In [ ]:
for time_label, time_info_dict in tqdm(split_dict.items()):
    pred_min = int(time_label[-1])
    gpr = load_model_gpr(
        name=f"gpr_{pred_min}", fp_checkpoints=fp_cur_model_folder)
    gpr_valid_pred = model_test_predictions_gpr(
        gpr=gpr, df_test=time_info_dict["valid_df"], pred_cols=time_info_dict["outputs"], 
        predictors=predictors, 
        regressor_label=f"_{time_label}", pred_min=pred_min, seed=seed)
    gpr_valid_pred.to_csv(join(fp_cur_predictions_folder, f"gpr_valid_{pred_min}.csv"))
    gpr_test_pred = model_test_predictions_gpr(
        gpr=gpr, df_test=time_info_dict["test_df"], pred_cols=time_info_dict["outputs"], 
        predictors=predictors, 
        regressor_label=f"_{time_label}", pred_min=pred_min, seed=seed)
    gpr_test_pred.to_csv(join(fp_cur_predictions_folder, f"gpr_test_{pred_min}.csv"))